## Neural network predicting 5 points at once based on velocities

In [1]:
from matplotlib import pyplot as plt
import numpy as np
np.random.seed(42)

## Data preparation

In [2]:
from data_processing import read_edinburgh_data 
# set sampling rate as 9 so that the velocities in the df are correct
data, agent_ids = read_edinburgh_data(sampling_rate=9)

/Users/kriis/miniforge3/envs/tensorflow_m1/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


reading:./OpenTraj/datasets/Edinburgh/annotations/tracks.01Sep.txt


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 2342/2342 [01:02<00:00, 37.51it/s]


Warning! too big dt in [Edinburgh]
finish


In [3]:
from sklearn.model_selection import train_test_split
train_agent_ids, test_agent_ids = train_test_split(agent_ids, test_size=0.2)

In [11]:
print(test_agent_ids)
np.save('test_agent_ids', test_agent_ids)

[1034 1858  763 2150  859 1347 2004  517  270 1288 2212  161  429  219
 1529 1722 1275  129  824  130  176 1231  266 1643   40  822  162  377
 1052 1706 1017 2120  672  730  775  953 1283  536 1385 2071 1480 2211
 1937  956 1442 1466  194  701  797  527  649 1820  501 1255 1374  843
 1198  769   51  912   55   52   71  450 1999 1209  901  578 1596 1060
 1153  915  505  747 1469  334 1233 1058 1548   97 1930  665  590 1615
  638 1716 2014  541 1244 1088  173   20  836 1118 1779 1238 1287   50
 1186 1513  128  858  788  531  165 1185 1713 1187  577  474 1978  160
 2094 1054  154 1602 1343  481  117 1920 1685  684 1460 1650   89 1836
 1076  461 1447 1651 1542   16  490  202  714 1720  737    2 1619 1291
 1748  506 1913 1201  589  504 1333  546 1538  163  366  778 1151  669
 1415]


### Helper functions

In [4]:
def create_dataset(data, agent_ids):
    data_X, data_Y_vel_x, data_Y_vel_y = [], [], []
    for agent_id in agent_ids:
        # We only see 4 velocities, so train is 4 velocities long
        x = data[data['agent_id'] == agent_id].iloc[1:5][['vel_x', 'vel_y']]
        y_vel_x = data[data['agent_id'] == agent_id].iloc[5:10]['vel_x']
        y_vel_y = data[data['agent_id'] == agent_id].iloc[5:10]['vel_y']
        data_X.append(x)
        data_Y_vel_x.append(y_vel_x)
        data_Y_vel_y.append(y_vel_y)
    return np.array(data_X), np.array(data_Y_vel_x), np.array(data_Y_vel_y)
    
def calculate_error(pred_x, pred_y, test_x, test_y):
    
    # FDE
    final_displacement_x = pred_x[-1] - test_x[-1]
    final_displacement_y = pred_y[-1] - test_y[-1]
    FDE = np.sqrt(final_displacement_x**2 + final_displacement_y**2)
    
    # MSE
    abs_error_x = pred_x - test_x
    abs_error_y = pred_y - test_y
    
    MSE = np.mean(np.square(abs_error_x)) + np.mean(np.square(abs_error_y))
    
    return FDE, MSE

## Experiment

In [5]:
train = create_dataset(data, train_agent_ids)
test = create_dataset(data, test_agent_ids)

In [6]:
from keras.models import Model
from keras.layers import *
from keras import backend as K

# Note: RMSE seems to yield a worse loss than mean_squared_error
def RMSE(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 
    
def RMSE_with_discount(y_true, y_pred):
    difference = y_true - y_pred
    difference = difference * [1, 0.95, 0.90, 0.85, 0.80]
    return K.sqrt(K.mean(K.square(difference))) 
    

#inp = Input((5,2))
inp = Input((4,2))

x = Flatten()(inp)

x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dense(16, activation='relu')(x)

'''
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
'''
out1 = Dense(5, activation='linear')(x)    
out2 = Dense(5, activation='linear')(x)

model = Model(inputs=inp, outputs=[out1,out2])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss = RMSE)

Metal device set to: Apple M1


2022-01-13 23:45:06.473205: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-13 23:45:06.473289: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
y_vel_x, y_vel_y = train[1], train[2] # separate the y data into x and y position columns
model.fit(train[0], [y_vel_x, y_vel_y], epochs=400, batch_size=32, verbose=2)
model.save('models/Edinburgh_NN_5_point_pred_velocity')

Epoch 1/400


2022-01-13 23:45:06.603195: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-13 23:45:06.797377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


20/20 - 0s - loss: 1.6012 - dense_4_loss: 0.7648 - dense_5_loss: 0.8364 - 456ms/epoch - 23ms/step
Epoch 2/400
20/20 - 0s - loss: 1.4524 - dense_4_loss: 0.7383 - dense_5_loss: 0.7141 - 114ms/epoch - 6ms/step
Epoch 3/400
20/20 - 0s - loss: 1.3032 - dense_4_loss: 0.6959 - dense_5_loss: 0.6074 - 107ms/epoch - 5ms/step
Epoch 4/400
20/20 - 0s - loss: 1.1959 - dense_4_loss: 0.6345 - dense_5_loss: 0.5613 - 117ms/epoch - 6ms/step
Epoch 5/400
20/20 - 0s - loss: 1.1108 - dense_4_loss: 0.5636 - dense_5_loss: 0.5472 - 120ms/epoch - 6ms/step
Epoch 6/400
20/20 - 0s - loss: 1.0395 - dense_4_loss: 0.5101 - dense_5_loss: 0.5294 - 118ms/epoch - 6ms/step
Epoch 7/400
20/20 - 0s - loss: 0.9750 - dense_4_loss: 0.4632 - dense_5_loss: 0.5118 - 117ms/epoch - 6ms/step
Epoch 8/400
20/20 - 0s - loss: 0.9301 - dense_4_loss: 0.4338 - dense_5_loss: 0.4963 - 112ms/epoch - 6ms/step
Epoch 9/400
20/20 - 0s - loss: 0.9160 - dense_4_loss: 0.4217 - dense_5_loss: 0.4944 - 122ms/epoch - 6ms/step
Epoch 10/400
20/20 - 0s - loss

2022-01-13 23:45:50.319814: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/Edinburgh_NN_5_point_pred_velocity/assets


In [8]:
trainScore = model.evaluate(train[0], [train[1], train[2]], verbose=0)
print('Train Score: %.2f' % (trainScore[0]))

testScore = model.evaluate(test[0], [test[1], test[2]], verbose=0)
print('Test Score: %.2f' % (testScore[0]))

Train Score: 0.57
Test Score: 1.09


2022-01-13 23:45:50.684402: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


## Making predictions on test data

In [ ]:
all_FDE = []
plotting = True
no_of_plotted_trajectories = 20

for idx, agent_id in enumerate(test_agent_ids):

    seen_x = np.array(data[data['agent_id'] == agent_id].iloc[:5]['pos_x'])
    seen_y = np.array(data[data['agent_id'] == agent_id].iloc[:5]['pos_y'])
    
    seen_vel_x = np.array(data[data['agent_id'] == agent_id].iloc[1:5]['vel_x'])
    seen_vel_y = np.array(data[data['agent_id'] == agent_id].iloc[1:5]['vel_y'])
    
    unseen_x = np.array(data[data['agent_id'] == agent_id].iloc[5:10]['pos_x'])
    unseen_y = np.array(data[data['agent_id'] == agent_id].iloc[5:10]['pos_y'])

    pred_vel_x, pred_vel_y = model.predict(np.array([np.column_stack((seen_vel_x, seen_vel_y))]))
    pred_vel_x = pred_vel_x[0] # destructure the prediction array as it is [[x1, x2, x3...]]
    pred_vel_y = pred_vel_y[0]
    
    pred_coord_x = [seen_x[-1] + pred_vel_x[0]]
    pred_coord_y = [seen_y[-1] + pred_vel_y[0]]
    
    for i in range(1, 5):
        pred_coord_x.append(pred_coord_x[-1] + pred_vel_x[i])
        pred_coord_y.append(pred_coord_y[-1] + pred_vel_y[i])

    FDE = np.sqrt((pred_coord_x[-1] - unseen_x[-1])**2 + (pred_coord_y[-1] - unseen_y[-1])**2)
    all_FDE.append(FDE)
    # plot data
    if plotting and idx < no_of_plotted_trajectories:
        plt.axes().set_aspect('equal')

        plt.plot(np.append(seen_x, unseen_x), np.append(seen_y, unseen_y), color='g')
        plt.scatter(np.append(seen_x, unseen_x), np.append(seen_y, unseen_y), color='g')

        plt.plot(pred_coord_x, pred_coord_y, color='r')
        plt.scatter(pred_coord_x, pred_coord_y, color='r')

        plt.scatter(seen_x[0], seen_y[0], color='b')

        plt.show()

print("Average FDE: ", np.mean(all_FDE))

In [ ]:
print(len(test_agent_ids))
print(len(train_agent_ids))

## Testing different hyperparameters

In [ ]:
# NN architecture
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

inp = Input((5,2))

x = Flatten()(inp)

x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dense(16, activation='relu')(x)

out1 = Dense(5, activation='linear')(x)    
out2 = Dense(5, activation='linear')(x)

In [ ]:
tf.get_logger().setLevel('ERROR')

learning_rates = [0.001, 0.005, 0.01]
batch_sizes = [1, 4, 8, 16, 32]
loss_functions = ['mean_squared_error', root_mean_squared_error]

all_train_RMSE = []
all_test_RMSE = []

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for loss_function in loss_functions:
            model = Model(inputs=inp, outputs=[out1,out2])
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss = loss_function)

            model.fit(train[0], [train[1], train[2]], epochs=200, batch_size=32, verbose=0)
            
            trainScore = model.evaluate(train[0], [train[1], train[2]], verbose=0)
            testScore = model.evaluate(test[0], [test[1], test[2]], verbose=0)
            
            if loss_function == 'mean_squared_error':
                all_train_RMSE.append(np.sqrt(trainScore[0]))
                all_test_RMSE.append(np.sqrt(testScore[0]))
                #Wprint("Learning rate: %.3f, batch size: %i, loss function: %s" % (learning_rate, batch_size, loss_function))
                #print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], np.sqrt(trainScore[0])))
                #print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], np.sqrt(testScore[0])))
            else:
                all_train_RMSE.append(trainScore[0])
                all_test_RMSE.append(testScore[0])
                #print("Learning rate: %.3f, batch size: %i, loss function: %s" % (learning_rate, batch_size, loss_function.__name__))
                #print('Train Score: %.2f RMSE' % (trainScore[0]))
                #print('Test Score: %.2f RMSE' % (testScore[0]))


In [ ]:
best_learning_rate = None
best_batch_size = None
best_loss_function = None
best_test_rmse = min(all_test_RMSE)
best_test_rmse_idx = all_test_RMSE.index(best_test_rmse)

rmse_idx = 0
for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for loss_function in loss_functions:
            
            if rmse_idx == best_test_rmse_idx:
                best_learning_rate = learning_rate
                best_batch_size = batch_size
                if loss_function == 'mean_squared_error':
                    best_loss_function = loss_function
                else:
                    best_loss_function = loss_function.__name__
            
            if loss_function == 'mean_squared_error':
                print("Learning rate: %.3f, batch size: %i, loss function: %s" % (learning_rate, batch_size, loss_function))
            else:
                print("Learning rate: %.3f, batch size: %i, loss function: %s" % (learning_rate, batch_size, loss_function.__name__))
            print('Train Score: %.2f RMSE' % (all_train_RMSE[rmse_idx]))
            print('Test Score: %.2f RMSE' % (all_test_RMSE[rmse_idx]))
            print()
            rmse_idx += 1

In [ ]:
print("Best test RMSE: ", best_test_rmse)
print("Best learning rate: ", best_learning_rate)
print("Best batch size: ", best_batch_size)
print("Best loss function: ", best_loss_function)